In [0]:
%python
# Cria o menu suspenso
dbutils.widgets.dropdown(
  "ano_escolhido", 
  "Todos", 
  ["Todos","2011", "2012"]
)

In [0]:

%sql
USE projeto_bikes_enterprise;

CREATE OR REPLACE TABLE gold_kpis_base AS
SELECT
  ano,
  mes,
  SUM(total_aluguel) AS quantidade, 
  ROUND(AVG(temperatura), 2) AS temperatura_media,
  SUM(CASE WHEN clima LIKE '%Chuva%' THEN 1 ELSE 0 END) AS dias_de_chuva
FROM
  silver_bikes_tratadas
GROUP BY
  ano, mes;

-------------- CAMADA GOLD - ANÁLISES/MÉTRICAS --------------
-- Qual foi o total de aluguéis por mês e qual a temperatura média?
-- Estamos melhorando ou piorando? Comparação com o mês anterior e Growth(a porcentagem)
-- Análise de crescimento MoM%
-- Qual o melhor mês da história?
-- Acumulado do ano. Meta batida de 2M por ano?

CREATE OR REPLACE TABLE gold_kpis AS
SELECT
  ano,
  mes,
  quantidade AS total_aluguel,
  temperatura_media,
  dias_de_chuva,
  
  -- Diferença
  quantidade - LAG(quantidade) OVER (ORDER BY ano, mes) as diferenca_mes_anterior, -- Comparação com o mês anterior
  
  -- Growth %
  ROUND(
        (quantidade - LAG(quantidade) OVER (ORDER BY ano, mes)) / 
        LAG(quantidade) OVER (ORDER BY ano, mes) * 100
    , 2) as pct_crescimento, -- Comparação %(Growth) com o mês anterior
  
  -- Ranking Global (pegar tudo)
  RANK () OVER(ORDER BY quantidade DESC) AS ranking,-- Qual foi o melhor mês da história?
  
  -- Acumulado
  SUM(quantidade) OVER(PARTITION BY ano ORDER BY mes) AS acumulado_ano, -- Acumulado no ano
  
  2000000 AS meta_anual,
  
  -- Status da Meta 
  CASE 
    WHEN SUM(quantidade) OVER(PARTITION BY ano ORDER BY mes) >= 2000000 THEN 'Batida'
    ELSE 'Não Batida'
  END AS status_meta,
  
  -- Barra de Progresso 
  ROUND((SUM(quantidade) OVER(PARTITION BY ano ORDER BY mes) / 2000000) * 100, 1) as pct_meta_concluida-- Percentual de quanto foi concluído da meta (Barra de progresso para PBI)

FROM
  gold_kpis_base
ORDER BY
  ano,
  mes ASC;

SELECT * FROM gold_kpis;

ano,mes,total_aluguel,temperatura_media,dias_de_chuva,diferenca_mes_anterior,pct_crescimento,ranking,acumulado_ano,meta_anual,status_meta,pct_meta_concluida
2011,1,38189,8.12,1,null,null,24,38189,2000000,Não Batida,1.9
2011,2,48215,11.59,0,10026,26.25,23,86404,2000000,Não Batida,4.3
2011,3,64045,13.59,2,15830,32.83,22,150449,2000000,Não Batida,7.5
2011,4,94870,19.31,1,30825,48.13,20,245319,2000000,Não Batida,12.3
2011,5,135821,23.67,0,40951,43.17,13,381140,2000000,Não Batida,19.1
2011,6,143512,28.42,0,7691,5.66,10,524652,2000000,Não Batida,26.2
2011,7,141341,31.1,0,-2171,-1.51,11,665993,2000000,Não Batida,33.3
2011,8,136691,28.92,0,-4650,-3.29,12,802684,2000000,Não Batida,40.1
2011,9,127418,25.13,3,-9273,-6.78,14,930102,2000000,Não Batida,46.5
2011,10,123511,19.27,3,-3907,-3.07,16,1053613,2000000,Não Batida,52.7


In [0]:
%sql
SELECT * 
FROM 
  gold_kpis 
WHERE
  (:ano_escolhido = 'Todos' OR CAST(ano AS STRING)= :ano_escolhido) 
ORDER BY 
  ano, 
  mes ASC;

ano,mes,total_aluguel,temperatura_media,dias_de_chuva,diferenca_mes_anterior,pct_crescimento,ranking,acumulado_ano,meta_anual,status_meta,pct_meta_concluida
2011,1,38189,8.12,1,null,null,24,38189,2000000,Não Batida,1.9
2011,2,48215,11.59,0,10026,26.25,23,86404,2000000,Não Batida,4.3
2011,3,64045,13.59,2,15830,32.83,22,150449,2000000,Não Batida,7.5
2011,4,94870,19.31,1,30825,48.13,20,245319,2000000,Não Batida,12.3
2011,5,135821,23.67,0,40951,43.17,13,381140,2000000,Não Batida,19.1
2011,6,143512,28.42,0,7691,5.66,10,524652,2000000,Não Batida,26.2
2011,7,141341,31.1,0,-2171,-1.51,11,665993,2000000,Não Batida,33.3
2011,8,136691,28.92,0,-4650,-3.29,12,802684,2000000,Não Batida,40.1
2011,9,127418,25.13,3,-9273,-6.78,14,930102,2000000,Não Batida,46.5
2011,10,123511,19.27,3,-3907,-3.07,16,1053613,2000000,Não Batida,52.7


In [0]:
%sql

------- TESTES PARA A QUALIDADE DOS DADOS -------
-- Se o resultado for vazio, está tudo perfeito.
-- Se aparecer alguma linha, tem um bug para corrigir.

-- TESTE 1: A meta concluída nunca pode ser negativa 
SELECT * FROM gold_kpis WHERE pct_meta_concluida < 0;

-- TESTE 2: Não pode haver mês duplicado no mesmo ano 
SELECT 
  ano, 
  mes, 
  count(*) 
FROM gold_kpis 
GROUP BY ano, mes 
HAVING count(*) > 1;

ano,mes,count(*)


In [0]:
%sql
-- Reorganiza os arquivos físicos para consultas ultra-rápidas por Ano e Mês
OPTIMIZE gold_kpis
ZORDER BY (ano, mes);

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 4212), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1767806149184, 1767806151734, 8, 0, null, List(0, 0), null, 12, 12, 0, 0, null)"


In [0]:
%sql
-- Adicionando descrições para o dicionário de dados
ALTER TABLE gold_kpis CHANGE COLUMN pct_crescimento COMMENT 'Indicador Growth: Comparação percentual com o mês anterior (MoM)';
ALTER TABLE gold_kpis CHANGE COLUMN status_meta COMMENT 'Regra de Negócio: A meta é considerada batida se atingir 2 Milhões';
ALTER TABLE gold_kpis CHANGE COLUMN ranking COMMENT 'Posição do mês em relação ao histórico total de vendas';

DESCRIBE EXTENDED gold_kpis;

col_name,data_type,comment
ano,int,null
mes,int,null
total_aluguel,bigint,null
temperatura_media,double,null
dias_de_chuva,bigint,null
diferenca_mes_anterior,bigint,null
pct_crescimento,double,Indicador Growth: Comparação percentual com o mês anterior (MoM)
ranking,int,Posição do mês em relação ao histórico total de vendas
acumulado_ano,bigint,null
meta_anual,int,null
